In [58]:
import numpy as np
from matplotlib import pyplot as plt

import time
import datetime
from config import *
from datetime import timedelta
from binance.client import Client
from BinanceFuturesPy.futurespy import Client as cf

In [59]:
client = Client(api_key, api_secret)
clientf = cf(api_key, api_secret)#, testnet=True)
clientf.change_leverage(35)

{'symbol': 'BTCUSDT', 'leverage': 45, 'maxNotionalValue': '1000000'}

In [60]:
def cheese(depth):
    # depth = client.futures_coin_order_book(symbol='BTCUSD_PERP', pair="B65TCBUSD")
    bid = np.array(depth['bids']).astype(np.float64)
    ask = np.array(depth['asks']).astype(np.float64)
    
    ### price
    b_price = bid[:, 0][0] # compra
    a_price = ask[:, 0][0] # venta
    price = (b_price+a_price)/2
    
    ### order book
    k = 400
    b_prc = bid[:, 0].astype(np.float64)[:k]
    b_ord = bid[:, 1].astype(np.float64)[:k]
    # b_prc = b_prc[b_ord>b_ord.mean()]
    b_vol = b_ord.sum()
    std_b = round(np.std(np.histogram(b_prc)[0]), 3)
    
    a_prc = ask[:, 0].astype(np.float64)[:k]
    a_ord = ask[:, 1].astype(np.float64)[:k]
    # a_prc = a_prc[a_ord>a_ord.mean()]
    a_vol = a_prc.sum()
    std_a = round(np.std(np.histogram(a_prc)[0]), 3)
    
#     total_vol = a_vol+b_vol
#     rel_a = a_vol/total_vol
#     rel_b = b_vol/total_vol
    
    std_b = round(std_b, 1)
    std_a = round(std_a, 1)
    
    # print(std_b, std_a)
    return std_b, std_a, b_vol, a_vol, price

In [61]:
def buy_order():
    order = clientf.new_order(
                        # symbol = 'BTCUSDT',
                        symbol = 'BTCBUSD',
                        side = 'BUY',
                        orderType = 'MARKET',
                        quantity = 1e-3,
                    )
    return order

def sell_order():
    order = clientf.new_order(
                        # symbol = 'BTCUSDT',
                        symbol = 'BTCBUSD',
                        side = 'SELL',
                        orderType = 'MARKET',
                        quantity = 1e-3,
                    )
    return order

def buy_order():
    return 1

def sell_order():
    return 0

In [66]:
i_3 = 0
activo = 0
on = 0
on2 = 0
side = ""
lado1 = 0
lado2 = 0
diff = 0
sl = []
record = []

acc = 0
acc2 = 0

b, a = 0, 0
profit = 0

diff2 = 0
profit = 0

while True:
    # print(b, a, diff, acc, acc2, end=' ')
    # t = "{} {:.1f} {:.1f} {:.1f} {} {:.3f} {:.3f}".format(i_3, b, a, diff, side, acc, acc2)
    t = "{} {:.1f} {:.1f} {:.1f} {:.4f}".format(i_3, b, a, diff2, profit)
    t = f"{i_3} {round(diff2, 3)} {profit}"
    print(t, end=" ")
    # print(1)
    # time.sleep(2)
    # price = client.futures_ticker(symbol='BTCUSDT')['lastPrice']
    try:
        # price = 48327.6
        depth = client.futures_coin_order_book(symbol='BTCUSD_PERP', pair="BTCBUSD")
        profit = float(clientf.account_info()['assets'][2]['unrealizedProfit'])
        # price = client.futures_coin_mark_price(symbol='BTCUSD_PERP')
        b, a, vol_b, vol_a, price = cheese(depth)

        diff = abs(b-a)
        diff2 = b-a
        diff/=2.7
        diff2/=2.7
        
        if diff>=.99 and on==0:
            i_3+=1
        elif diff>=.99 and on==1:
            i_3+=1
        else:
            if i_3>0:
                i_3-=1

        if i_3>=2 and activo==0:
            activo=1

        if b>a:
            lado2 = 1
        elif a>b:
            lado2 = -1

        ###1~apertura
        if activo==1 and on==0:
            if b>a:
                ### venta
                order = sell_order()
                side = "sell"
                lado1 = 1
            elif a>b:
                ### compra
                order = buy_order()
                side = 'buy'
                lado1 = -1
            on=1
            print("open", side)
        elif activo==0 and on==1 and side!="":
            if side=="sell":
                order = buy_order()
            else:
                order = sell_order()
            i_3 = 0
            activo = 0
            on = 0
            side = ''
            sl = []
            record = []
            print("close 1")
        
        ###2~cierre
        elif activo==1 and on==1 and lado1!=lado2 and side!="":
            if side=="sell":
                order = buy_order()
            elif side=="buy":
                order = sell_order()
            i_3 = 0
            activo = 0
            on = 0
            side = ''
            sl = []
            record = []
            print("close 2")

        ###3~SL&TP
#         if on==1:
#             record.append(profit)
#         if len(record)>1:
#             acc = record[-1]
#         else:
#             acc = 0
        ###SL
#         if profit>0:
#             acc2 = profit*.75
#             sl.append(acc2)
#         else:
#             acc=0
        if profit>0:
            acc2 = profit*.75
            sl.append(acc2)
        ###SL - ojo
        if profit<=-0.06:
            if side=='buy':
                order = sell_order()
                i_3 = 0
                activo = 0
                on = 0
                side = ""
                sl = []
                record = []
                print("close 3")
                
            elif side=="sell":
                order = buy_order()
                i_3 = 0
                activo = 0
                on = 0
                side = ""
                sl = []
                record = []
                print("close 4")
        ##TP
        if len(sl)>=4:
            stop = max(sl)
            if side=='buy':
                if profit<stop:
                    order = sell_order()
                    i_3 = 0
                    activo = 0
                    on = 0
                    side = ""
                    sl = []
                    record = []
                    print("close 5")
            elif side=="sell":
                if profit<stop:
                    order = buy_order()
                    i_3 = 0
                    activo = 0
                    on = 0
                    side = ""
                    sl = []
                    record = []
                    print("close 6")

    # except requests.exceptions.ConnectTimeout:
    except SyntaxError:
        #print(".", end=" ")
        print("timeout")
        if activo==1 and on==1:
            if side=="sell":
                order = buy_order()
            elif side=="buy":
                order = sell_order()
            else:
                pass
            i_3 = 0
            activo = 0
            on = 0
            side = ''
            sl = []
            record = []
            print("close 7")
        time.sleep(15)
    # i_3, b, a, diff, side, acc, acc2
    print()

0 0 0 
0 0.444 0.0 
0 0.852 0.0 
1 1.556 0.0 open sell

2 1.111 0.0 
3 1.444 0.0 
4 1.407 0.0 
5 2.259 0.0 
4 0.815 0.0 
5 1.556 0.0 
6 1.556 0.0 
7 1.37 0.0 
8 1.333 0.0 
9 1.704 0.0 
8 0.741 0.0 
7 0.63 0.0 
8 1.444 0.0 
7 0.593 0.0 
6 0.481 0.0 
5 0.889 0.0 
4 0.778 0.0 
3 0.963 0.0 close 2

0 -0.111 0.0 
0 0.333 0.0 
0 0.778 0.0 
1 1.185 0.0 
0 0.852 0.0 
0 0.926 0.0 
1 1.0 0.0 
0 0.741 0.0 
1 2.296 0.0 open sell

2 1.111 0.0 
3 1.519 0.0 
4 1.481 0.0 close 2

0 -0.111 0.0 
1 -1.259 0.0 
0 -0.259 0.0 
0 -0.667 0.0 
0 0.296 0.0 
0 0.185 0.0 
0 0.074 0.0 
0 0.0 0.0 
1 1.963 0.0 open sell

2 1.667 0.0 
3 2.259 0.0 
4 2.222 0.0 
5 1.704 0.0 
6 1.926 0.0 
7 4.519 0.0 
8 3.519 0.0 
9 1.333 0.0 
10 1.111 0.0 
11 1.889 0.0 
12 2.148 0.0 
13 1.852 0.0 
14 1.778 0.0 
15 1.593 0.0 
16 2.222 0.0 
17 1.63 0.0 close 2

0 -1.741 0.0 
0 0.259 0.0 
0 0.074 0.0 
0 0.481 0.0 
1 1.407 0.0 
0 0.519 0.0 
1 1.296 0.0 
0 0.778 0.0 
0 0.481 0.0 
0 0.481 0.0 
0 0.556 0.0 
0 0.37 0.0 
1 1.593 0.0 open sell



2 1.185 0.0 
3 2.667 0.0 
4 1.667 0.0 
5 1.778 0.0 
6 1.778 0.0 
7 1.778 0.0 
8 1.704 0.0 
9 1.667 0.0 
10 1.963 0.0 
11 1.296 0.0 
10 0.407 0.0 
9 0.889 0.0 
10 2.296 0.0 
11 2.778 0.0 
12 2.889 0.0 
13 3.037 0.0 
14 3.111 0.0 
15 2.852 0.0 
16 2.815 0.0 
17 2.222 0.0 
18 1.667 0.0 
19 2.852 0.0 
20 3.963 0.0 
21 3.519 0.0 
22 2.889 0.0 
23 1.741 0.0 
24 1.444 0.0 
25 1.444 0.0 
26 3.074 0.0 
27 2.593 0.0 
28 3.296 0.0 
29 3.185 0.0 
30 2.815 0.0 
31 1.556 0.0 
30 0.815 0.0 
31 1.556 0.0 
32 5.0 0.0 
33 4.222 0.0 
34 2.852 0.0 
35 3.741 0.0 
36 3.704 0.0 
37 2.889 0.0 
38 1.333 0.0 
39 1.37 0.0 
40 1.074 0.0 
41 1.185 0.0 
40 0.963 0.0 
41 1.0 0.0 
42 4.407 0.0 
41 0.148 0.0 
40 0.593 0.0 
39 0.667 0.0 
38 0.185 0.0 close 2

0 -0.074 0.0 
0 0.074 0.0 
0 -0.185 0.0 
0 0.111 0.0 
0 0.63 0.0 
0 0.889 0.0 
1 1.259 0.0 open sell

2 1.037 0.0 
3 1.741 0.0 
4 1.481 0.0 
5 1.111 0.0 
4 0.185 0.0 
3 0.889 0.0 
2 0.185 0.0 
3 3.667 0.0 
4 2.593 0.0 
5 2.148 0.0 
6 2.0 0.0 
7 2.0 0.0 
8 1.444 0.

44 4.111 0.0 
45 2.407 0.0 
46 3.37 0.0 
47 3.704 0.0 
48 3.667 0.0 
49 3.259 0.0 
50 3.37 0.0 
51 2.185 0.0 
52 2.815 0.0 
53 2.815 0.0 
54 2.63 0.0 
55 1.926 0.0 
54 0.556 0.0 
53 0.333 0.0 
54 1.111 0.0 
55 1.222 0.0 
56 1.111 0.0 
57 1.074 0.0 
58 3.519 0.0 
59 3.556 0.0 
60 3.963 0.0 
61 3.963 0.0 close 2

0 -4.259 0.0 
1 -1.778 0.0 open buy

2 -2.667 0.0 
1 -0.889 0.0 
0 -0.889 0.0 
1 -2.037 0.0 
2 -1.963 0.0 
1 -0.741 0.0 
2 -1.444 0.0 close 2

0 1.519 0.0 
0 0.0 0.0 
0 0.037 0.0 
0 0.259 0.0 
0 0.519 0.0 
0 0.333 0.0 
1 1.519 0.0 open sell

2 1.333 0.0 
1 0.259 0.0 close 2

0 -1.407 0.0 
1 -1.259 0.0 
0 -0.667 0.0 
0 -0.37 0.0 
0 -0.926 0.0 
0 -0.926 0.0 
1 -1.333 0.0 open buy

2 -1.519 0.0 
3 -1.37 0.0 
4 -1.111 0.0 
3 -0.444 0.0 
4 -1.556 0.0 
5 -1.111 0.0 close 2

0 2.074 0.0 
1 3.148 0.0 open sell

2 2.259 0.0 
3 2.222 0.0 
4 2.741 0.0 
5 2.704 0.0 
6 2.741 0.0 
7 2.741 0.0 
8 2.741 0.0 
9 2.778 0.0 
10 3.185 0.0 
11 3.0 0.0 
12 2.741 0.0 
13 2.778 0.0 
14 2.519 0.0 
15 2.6

33 1.37 0.0 
32 0.741 0.0 
33 1.074 0.0 
34 1.074 0.0 
35 1.222 0.0 
36 1.111 0.0 
37 1.667 0.0 
38 1.37 0.0 close 2

0 -0.185 0.0 
0 -0.037 0.0 
0 -0.148 0.0 
0 0.407 0.0 
0 0.444 0.0 
0 0.111 0.0 
0 0.333 0.0 
0 0.333 0.0 
1 1.0 0.0 
0 0.63 0.0 
1 1.0 0.0 open sell

2 1.185 0.0 
1 0.815 0.0 
0 0.741 0.0 
1 1.222 0.0 
2 1.222 0.0 
3 1.407 0.0 
2 0.963 0.0 
1 0.963 0.0 
0 0.481 0.0 
0 0.667 0.0 
1 1.667 0.0 
2 1.333 0.0 
3 1.074 0.0 
2 0.926 0.0 
3 1.185 0.0 
2 0.519 0.0 
3 1.074 0.0 
4 1.037 0.0 
3 0.111 0.0 close 2

0 -0.037 0.0 
0 -0.481 0.0 
0 -0.333 0.0 
0 0.556 0.0 
0 0.704 0.0 
0 0.556 0.0 
0 0.852 0.0 
1 1.37 0.0 open sell

2 1.185 0.0 
3 1.556 0.0 
4 1.556 0.0 
5 4.074 0.0 
6 2.481 0.0 
7 2.074 0.0 
8 2.407 0.0 
9 2.037 0.0 
10 2.111 0.0 
11 2.0 0.0 
12 1.333 0.0 
13 1.296 0.0 
14 1.444 0.0 
13 0.778 0.0 
12 0.704 0.0 
13 1.519 0.0 
14 2.741 0.0 
15 2.111 0.0 
16 2.074 0.0 
17 2.111 0.0 
18 2.296 0.0 
19 2.407 0.0 
20 1.889 0.0 
21 2.074 0.0 
22 1.963 0.0 
23 2.148 0.0 
24 3.7

86 -1.148 0.0 
87 -1.296 0.0 
88 -1.37 0.0 
87 -0.963 0.0 
88 -1.148 0.0 
89 -1.111 0.0 
90 -1.519 0.0 
91 -1.556 0.0 
92 -1.667 0.0 
93 -1.185 0.0 
92 -0.889 0.0 
93 -1.593 0.0 
94 -1.519 0.0 
95 -1.037 0.0 
96 -1.259 0.0 
97 -1.519 0.0 
98 -1.185 0.0 
99 -1.185 0.0 
100 -1.185 0.0 
101 -1.481 0.0 
102 -1.37 0.0 
103 -1.185 0.0 
104 -1.111 0.0 
103 -0.667 0.0 
102 -0.37 0.0 
101 -0.444 0.0 
100 -0.407 0.0 
101 -1.407 0.0 
102 -1.148 0.0 
103 -2.259 0.0 
104 -2.296 0.0 
105 -2.037 0.0 
106 -2.259 0.0 
107 -2.222 0.0 
108 -1.815 0.0 
109 -1.778 0.0 
110 -2.148 0.0 
111 -2.778 0.0 
112 -2.852 0.0 
113 -1.889 0.0 
114 -2.037 0.0 
115 -1.704 0.0 
116 -1.63 0.0 
117 -1.704 0.0 
118 -1.778 0.0 
119 -2.815 0.0 
120 -1.889 0.0 
121 -1.815 0.0 
122 -2.63 0.0 
123 -1.667 0.0 
122 -0.963 0.0 
121 -0.778 0.0 
122 -1.074 0.0 
121 -0.444 0.0 
120 -0.667 0.0 
121 -1.407 0.0 
120 -0.963 0.0 
119 -0.667 0.0 
120 -1.074 0.0 
121 -1.519 0.0 
122 -1.333 0.0 
123 -1.556 0.0 
124 -1.741 0.0 
125 -1.63 0.0 


14 -1.222 0.0 
15 -1.704 0.0 
14 -0.704 0.0 
13 -0.815 0.0 
12 -0.741 0.0 close 2

0 0.519 0.0 
0 0.815 0.0 
0 0.296 0.0 
0 0.296 0.0 
0 0.111 0.0 
0 -0.037 0.0 
0 0.556 0.0 
0 0.074 0.0 
0 0.148 0.0 
0 0.185 0.0 
0 -0.222 0.0 
0 -0.556 0.0 
0 0.185 0.0 
0 -0.148 0.0 
0 -0.037 0.0 
0 -0.407 0.0 
0 -0.185 0.0 
0 0.148 0.0 
0 0.074 0.0 
0 0.185 0.0 
0 -0.037 0.0 
0 -0.111 0.0 
0 -0.074 0.0 
0 -0.148 0.0 
0 -0.185 0.0 
0 0.185 0.0 
0 0.074 0.0 
0 -0.222 0.0 
0 0.037 0.0 
1 1.074 0.0 
0 -0.074 0.0 
0 0.407 0.0 
0 0.407 0.0 
0 0.037 0.0 
0 0.407 0.0 
1 1.185 0.0 
0 0.778 0.0 
1 1.074 0.0 
0 0.741 0.0 
1 1.111 0.0 open sell

2 1.0 0.0 
1 0.259 0.0 
0 0.556 0.0 
0 0.481 0.0 
0 0.259 0.0 
0 0.741 0.0 
0 0.778 0.0 
0 0.704 0.0 
0 0.778 0.0 
1 1.333 0.0 
2 1.111 0.0 
1 0.889 0.0 
0 0.926 0.0 
0 0.815 0.0 
0 0.63 0.0 
0 0.815 0.0 
0 0.519 0.0 
0 0.852 0.0 
0 0.407 0.0 
0 0.556 0.0 
1 2.037 0.0 
2 2.556 0.0 
3 2.778 0.0 
4 1.667 0.0 
5 2.148 0.0 
6 2.296 0.0 
7 2.074 0.0 
8 2.074 0.0 
9 1.63 0.0 


0 0.148 0.0 
0 0.222 0.0 
0 0.0 0.0 
0 0.852 0.0 
0 0.704 0.0 
0 0.852 0.0 
1 1.185 0.0 open sell

2 2.074 0.0 
3 2.296 0.0 
2 0.259 0.0 
1 0.259 0.0 close 2

0 -1.148 0.0 
1 -1.778 0.0 open buy

2 -3.444 0.0 
3 -3.296 0.0 
2 -0.407 0.0 
1 -0.296 0.0 
0 -0.259 0.0 
1 -1.519 0.0 
2 -1.296 0.0 
3 -2.0 0.0 
4 -1.852 0.0 
5 -1.0 0.0 
6 -1.037 0.0 
5 -0.815 0.0 
6 -1.148 0.0 
7 -1.037 0.0 
6 -0.963 0.0 
7 -1.667 0.0 
6 -0.222 0.0 
7 -1.111 0.0 
6 -0.407 0.0 
5 -0.111 0.0 
6 -1.148 0.0 
5 -0.815 0.0 
4 -0.889 0.0 
5 -1.444 0.0 
6 -1.0 0.0 
5 -0.963 0.0 
6 -1.0 0.0 
5 -0.889 0.0 
4 -0.481 0.0 
3 -0.074 0.0 close 2

0 0.111 0.0 
0 -0.37 0.0 
0 -0.444 0.0 
0 -0.852 0.0 
1 -1.185 0.0 open buy

2 -1.296 0.0 
3 -1.185 0.0 close 2

0 0.852 0.0 
0 0.259 0.0 
0 0.667 0.0 
0 0.556 0.0 
0 0.519 0.0 
1 -3.0 0.0 open buy

2 -1.185 0.0 
3 -1.333 0.0 
2 -0.63 0.0 
1 -0.852 0.0 close 2

0 0.148 0.0 
0 -0.148 0.0 
0 0.259 0.0 
0 -0.222 0.0 
0 -0.593 0.0 
0 -0.63 0.0 
0 -0.222 0.0 
0 -0.815 0.0 
0 -0.778 0.0 

7 2.852 0.0 
8 2.63 0.0 
9 3.407 0.0 
10 3.296 0.0 
11 3.37 0.0 
12 3.0 0.0 
13 3.0 0.0 close 2

0 -0.185 0.0 
0 0.111 0.0 
1 2.074 0.0 open sell

2 1.556 0.0 
3 1.926 0.0 
4 2.852 0.0 
5 1.037 0.0 
6 1.222 0.0 
5 0.148 0.0 
4 0.852 0.0 
3 0.926 0.0 
2 0.407 0.0 
1 0.074 0.0 
0 0.111 0.0 close 2

0 -0.444 0.0 
0 -0.148 0.0 
0 0.296 0.0 
0 0.185 0.0 
0 0.0 0.0 
0 -0.259 0.0 
1 1.704 0.0 open sell

2 1.704 0.0 
3 2.037 0.0 
4 1.889 0.0 
5 2.037 0.0 
6 1.074 0.0 
7 1.667 0.0 
8 1.333 0.0 
9 1.111 0.0 
8 0.222 0.0 close 2

0 -0.815 0.0 
1 -1.222 0.0 open buy

2 -1.185 0.0 
3 -1.0 0.0 
4 -1.074 0.0 
5 -1.593 0.0 
6 -1.593 0.0 
7 -1.407 0.0 
6 -0.926 0.0 
7 -1.296 0.0 
6 -0.926 0.0 
7 -1.37 0.0 
6 -0.407 0.0 
5 -0.556 0.0 
6 -1.37 0.0 
5 -0.407 0.0 
4 -0.074 0.0 
3 -0.333 0.0 
2 -0.519 0.0 
1 -0.148 0.0 
0 -0.185 0.0 
0 -0.444 0.0 close 2

0 0.185 0.0 
0 -0.148 0.0 
0 -0.185 0.0 
1 -1.0 0.0 open buy

2 -1.741 0.0 
3 -1.222 0.0 
2 -0.889 0.0 
1 -0.519 0.0 close 2

0 0.259 0.0 
0 0.444 0.0 
0 

KeyboardInterrupt: 